# Scraping
#### Se realiza la carga de datos desde el sitio argenprop.com 

### Intalación de las librerías necesarias

In [69]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [70]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [71]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


### Importamos las librerías y creamos el dataset de Pandas

In [72]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm.auto import tqdm
import os

df=pd.DataFrame(columns=['Precio','Ubicacion','Barrio','Metros','Dorm','Baños','Años'])

### Obtenemos el número total de páginas de publicaciones

In [73]:
payload = {'api_key': 'e2cfbc1810d6a0321cfc04c45c315c05', 'url': 'https://www.argenprop.com/departamento-venta-region-capital-federal-1-o-más-cocheras-pagina-2'}
r = requests.get('http://api.scraperapi.com', params=payload)
soup = BeautifulSoup(r.text, 'lxml')
paginas = soup.find_all('li', class_="pagination__page")
paginas = int(paginas[-2].text)
print("Total de páginas:",paginas)

Total de páginas: 856


### Recolectamos la información de cada página, la parseamos y la incuimos en el dataset

In [79]:

for j in tqdm(range(2,paginas)): # PARA MENOR TIEMPO reemplazar paginas por otro valor así no se trae la totalidad de páginas 
    payload = {'api_key': 'e2cfbc1810d6a0321cfc04c45c315c05', 'url': 'https://www.argenprop.com/departamento-venta-region-capital-federal-1-o-más-cocheras-pagina-2'}
    r = requests.get('http://api.scraperapi.com', params=payload)
    
    soup = BeautifulSoup(r.text, 'lxml')

    precio = soup.find_all('p',class_="card__price")
    caract = soup.find_all('ul',class_="card__main-features")
    direccion = soup.find_all('p',class_="card__title--primary")

    try:
        ind = df.index[-1]+1
    except: 
        ind = 0

    for i in range(len(precio)):

        precios = precio[i].text.replace("USD ","").replace("$ ","").split()
        caracts = caract[i].text.split()
        direcciones = direccion[i].text.replace("Departamento en Venta en ","").split(", ")
        
        df.loc[ind+i,'Ubicacion'] = direcciones[0]
        df.loc[ind+i,'Barrio'] = direcciones[1]
        
        if precios[0] == 'Consultar' :
            df.loc[ind+i,'Precio'] = 999
        else:
            df.loc[ind+i,'Precio'] = int(precios[0].replace('.',''))

        if 'm²' in caracts:
            df.loc[ind+i,'Metros'] = float(caracts[caracts.index('m²')-1].replace(',','.'))

        if 'baño' in caracts:
            df.loc[ind+i,'Baños'] = int(caracts[caracts.index('baño')-1])

        if 'baños' in caracts:
            df.loc[ind+i,'Baños'] = int(caracts[caracts.index('baños')-1])

        if 'Monoam.' in caracts:
            df.loc[ind+i,'Dorm'] = 0
    
        if 'dorm.' in caracts:
            df.loc[ind+i,'Dorm'] = int(caracts[caracts.index('dorm.')-1])

        if 'años' in caracts:
            df.loc[ind+i,'Años'] = int(caracts[caracts.index('años')-1])

        if 'Estrenar' in caracts:
            df.loc[ind+i,'Años'] = 0
        

  0%|          | 0/854 [00:00<?, ?it/s]

[<p class="card__price">
<span class="card__currency">USD</span> 600.000            </p>, <p class="card__price">
<span class="card__currency">USD</span> 310.000            </p>, <p class="card__price">
<span class="card__currency">USD</span> 245.000            </p>, <p class="card__price">
<span class="card__currency">USD</span> 395.000            </p>, <p class="card__price">
<span class="card__currency">USD</span> 580.000            </p>, <p class="card__price">
<span class="card__currency">USD</span> 83.000            </p>, <p class="card__price">
<span class="card__currency">USD</span> 390.000            </p>, <p class="card__price">
<span class="card__currency">USD</span> 490.000            </p>, <p class="card__price">
<span class="card__currency">USD</span> 129.000            </p>, <p class="card__price">
<span class="card__currency">USD</span> 99.000            </p>, <p class="card__price">
<span class="card__currency">USD</span> 320.000            </p>, <p class="card__price"

KeyboardInterrupt: 

In [75]:
import findspark

findspark.add_jars('/app/postgresql-42.1.4.jar')
findspark.init()

In [76]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("pyspark-postgres")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

### Convertimos los valores faltantes en '999' y luego transformamos el Dataframe de Pandas en Spark

In [77]:
df=df.fillna(999)
df = spark.createDataFrame(df)

ValueError: can not infer schema from empty dataset

### Guardamos en Postgres

In [ ]:
df \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/workshop") \
    .option("dbtable", "workshop.viviendas") \
    .option("user", "workshop") \
    .option("password", "w0rkzh0p") \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [ ]:
df_pos = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/workshop") \
    .option("dbtable", "workshop.viviendas") \
    .option("user", "workshop") \
    .option("password", "w0rkzh0p") \
    .option("driver", "org.postgresql.Driver") \
    .load()

### Corroboramos que coincidan la cantidad de filas y columnas y mostramos las primeras 100 filas del Dataframe importado de postgres

In [ ]:
print((df.count(), len(df.columns)))
print((df_pos.count(), len(df_pos.columns)))

df_pos.show(100)

In [ ]:
spark.stop()